<a id='s8'></a>
# Rate of Reforestation widget

Dynamic Sentence based on FAO data in a Carto table. Only works for admin-0 (countries). Also ranks countries by rate of regrowth (ha/year) or relative rate (% of extent / year) and displays the country in a list similarly ranked countries. (e.g. if the selected country is ranked 5th, the list should show 3rd, 4th, 5th, 6th, 7th)


e.g. dynamic sentences:

"Between `X` and `Y` years, the rate of reforestation in `country name` was `X` ha/year."


** Options**

Users should be able to select period from a dropdown menu in the settings:
* Period: ['1990–2000', '2000–2005', '2005–2010', '2010–2015']

** Notes**

- Default should be the most recent period (`2010–2015`).
- The value that comes back from the query is kha not ha : i.e. it is in ha/1000 by default.

In [1]:
#Import Global Metadata etc

%run '0.Importable_Globals.ipynb'

In [2]:
# Human readable option for the Period selector, and a key that it should relate to for its query.
fao_periods = {'1990–2000': 1990, '2000–2005':2000, '2005–2010':2005, '2010–2015':2010}


# Variables
adm0 = 'BRA'
period = '2010–2015'


tags = ["forest_change"]
selectable_polynames = []

In [3]:
# First, get the FAO data from a carto table. Note that extent is multiplied by 1000.

sql = ("SELECT country AS iso, name, year, reforest AS rate, forest*1000 AS extent "
         "FROM table_1_forest_area_and_characteristics as fao "
         f"WHERE fao.year = {fao_periods[period]} "
         f"AND fao.country = '{adm0}'"
        )

account = 'wri-01'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": sql}
r = requests.get(urlCarto, params=sql)
print(r.url,'\n')
pprint(r.json())

data = r.json().get('rows')[0]

https://wri-01.carto.com/api/v2/sql?q=SELECT+country+AS+iso%2C+name%2C+year%2C+reforest+AS+rate%2C+forest%2A1000+AS+extent+FROM+table_1_forest_area_and_characteristics+as+fao+WHERE+fao.year+%3D+2010+AND+fao.country+%3D+%27BRA%27 

{'fields': {'extent': {'type': 'number'},
            'iso': {'type': 'string'},
            'name': {'type': 'string'},
            'rate': {'type': 'number'},
            'year': {'type': 'number'}},
 'rows': [{'extent': 498458000,
           'iso': 'BRA',
           'name': 'Brazil',
           'rate': 519,
           'year': 2010}],
 'time': 0.557,
 'total_rows': 1}


In [4]:
#Dynamic sentence

print(f"From {period}, the rate of reforestation in {data['name']} was {data['rate']*1000:,.0f} ha/year. ", end="")

From 2010–2015, the rate of reforestation in Brazil was 519,000 ha/year. 

### Rank countries by rate of reforestation (ha/year)

In [5]:
## RANK BY RATE

sql = ("WITH mytable AS (SELECT country AS iso, name, year, reforest AS rate, forest*1000 AS extent "
        "FROM table_1_forest_area_and_characteristics as fao "
        f"WHERE fao.year = {fao_periods[period]}),"
        "rank AS ( "
        "SELECT rate ,iso "
        "from mytable "
        "ORDER BY rate DESC), "
        f"item as (select rate from rank where iso = '{adm0}') "
        "select count(*) as rank "
        "from rank "
        "WHERE rate > (select rate from item) ")

account = 'wri-01'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": sql}
r = requests.get(urlCarto, params=sql)
print(r.url,'\n')
pprint(r.json())

rank = r.json().get('rows')[0].get('rank') + 1
print(rank)

https://wri-01.carto.com/api/v2/sql?q=WITH+mytable+AS+%28SELECT+country+AS+iso%2C+name%2C+year%2C+reforest+AS+rate%2C+forest%2A1000+AS+extent+FROM+table_1_forest_area_and_characteristics+as+fao+WHERE+fao.year+%3D+2010%29%2Crank+AS+%28+SELECT+rate+%2Ciso+from+mytable+ORDER+BY+rate+DESC%29%2C+item+as+%28select+rate+from+rank+where+iso+%3D+%27BRA%27%29+select+count%28%2A%29+as+rank+from+rank+WHERE+rate+%3E+%28select+rate+from+item%29+ 

{'fields': {'rank': {'type': 'number'}},
 'rows': [{'rank': 6}],
 'time': 0.002,
 'total_rows': 1}
7


In [12]:
## RANK BY RATE

sql = ("SELECT country AS iso, name, year, reforest AS rate, forest*1000 AS extent "
       "FROM table_1_forest_area_and_characteristics as fao "
       f"WHERE fao.year = {fao_periods[period]} "
       f"AND reforest > 0 "
       "ORDER BY rate DESC")

account = 'wri-01'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": sql}
r = requests.get(urlCarto, params=sql)
print(r.url,'\n')

rank_list = r.json().get('rows')

rank_list[0:3]

https://wri-01.carto.com/api/v2/sql?q=SELECT+country+AS+iso%2C+name%2C+year%2C+reforest+AS+rate%2C+forest%2A1000+AS+extent+FROM+table_1_forest_area_and_characteristics+as+fao+WHERE+fao.year+%3D+2010+AND+reforest+%3E+0+ORDER+BY+rate+DESC 



[{'extent': 13039200,
  'iso': 'MNG',
  'name': 'Mongolia',
  'rate': 9167,
  'year': 2010},
 {'extent': 20082012,
  'iso': 'SDN',
  'name': 'Sudan',
  'rate': 5940,
  'year': 2010},
 {'extent': 2605000,
  'iso': 'CRI',
  'name': 'Costa Rica',
  'rate': 4130.96,
  'year': 2010}]

In [7]:
if rank == 1:
    bottom_bound = 1
    upper_bound = 4
elif rank == 2:
    bottom_bound = 2
    upper_bound = 3    
elif rank == len(rank_list):
    bottom_bound = 5
    upper_bound = -1   
elif rank == len(rank_list)-1:
    bottom_bound = 4
    upper_bound = 0 
else:
    bottom_bound = 3
    upper_bound = 2 
    
rank_list[rank-bottom_bound:rank+upper_bound]

[{'extent': 815135600,
  'iso': 'RUS',
  'name': 'Russian Federation',
  'rate': 1018.7,
  'year': 2010},
 {'extent': 94432000,
  'iso': 'IDN',
  'name': 'Indonesia',
  'rate': 866.74,
  'year': 2010},
 {'extent': 498458000,
  'iso': 'BRA',
  'name': 'Brazil',
  'rate': 519,
  'year': 2010},
 {'extent': 347302000,
  'iso': 'CAN',
  'name': 'Canada',
  'rate': 401,
  'year': 2010},
 {'extent': 200610300,
  'iso': 'CHN',
  'name': 'China',
  'rate': 290.1,
  'year': 2010}]

In [8]:
### Rank countries by relative rate of reforestation (%/year)

In [9]:
## RANK BY RATE RELATIVE TO EXTENT

sql = ("WITH mytable AS (SELECT country AS iso, name, year, reforest AS rate, forest*1000 AS extent "
        "FROM table_1_forest_area_and_characteristics as fao "
        f"WHERE fao.year = {fao_periods[period]} "
        "AND forest > 0),"
        "rank AS ( "
        "SELECT rate, extent ,iso "
        "from mytable "
        "ORDER BY (rate/extent) DESC), "
        f"item as (select rate, extent from rank where iso = '{adm0}') "
        "select count(*) as rank "
        "from rank "
        "WHERE (rate/extent) > (select (rate/extent) from item) ")

account = 'wri-01'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": sql}
r = requests.get(urlCarto, params=sql)
print(r.url,'\n')
pprint(r.json())

rank = r.json().get('rows')[0].get('rank') + 1
print(rank)

https://wri-01.carto.com/api/v2/sql?q=WITH+mytable+AS+%28SELECT+country+AS+iso%2C+name%2C+year%2C+reforest+AS+rate%2C+forest%2A1000+AS+extent+FROM+table_1_forest_area_and_characteristics+as+fao+WHERE+fao.year+%3D+2010+AND+forest+%3E+0%29%2Crank+AS+%28+SELECT+rate%2C+extent+%2Ciso+from+mytable+ORDER+BY+%28rate%2Fextent%29+DESC%29%2C+item+as+%28select+rate%2C+extent+from+rank+where+iso+%3D+%27BRA%27%29+select+count%28%2A%29+as+rank+from+rank+WHERE+%28rate%2Fextent%29+%3E+%28select+%28rate%2Fextent%29+from+item%29+ 

{'fields': {'rank': {'type': 'number'}},
 'rows': [{'rank': 68}],
 'time': 0.002,
 'total_rows': 1}
69


In [13]:
## RANK BY RATE RELATIVE TO EXTENT

sql = ("SELECT country AS iso, name, year, reforest AS rate, forest*1000 AS extent "
       "FROM table_1_forest_area_and_characteristics as fao "
       f"WHERE fao.year = {fao_periods[period]} "
       f"AND reforest > 0 "
       "ORDER BY (reforest/forest) DESC")

account = 'wri-01'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": sql}
r = requests.get(urlCarto, params=sql)
print(r.url,'\n')

rank_list = r.json().get('rows')
rank_list[0:3]

https://wri-01.carto.com/api/v2/sql?q=SELECT+country+AS+iso%2C+name%2C+year%2C+reforest+AS+rate%2C+forest%2A1000+AS+extent+FROM+table_1_forest_area_and_characteristics+as+fao+WHERE+fao.year+%3D+2010+AND+reforest+%3E+0+ORDER+BY+%28reforest%2Fforest%29+DESC 



[{'extent': 2605000,
  'iso': 'CRI',
  'name': 'Costa Rica',
  'rate': 4130.96,
  'year': 2010},
 {'extent': 13039200,
  'iso': 'MNG',
  'name': 'Mongolia',
  'rate': 9167,
  'year': 2010},
 {'extent': 20082012,
  'iso': 'SDN',
  'name': 'Sudan',
  'rate': 5940,
  'year': 2010}]

In [11]:
if rank == 1:
    bottom_bound = -1
    upper_bound = 4
elif rank == 2:
    bottom_bound = 2
    upper_bound = 3    
elif rank == len(rank_list):
    bottom_bound = 5
    upper_bound = -1   
elif rank == len(rank_list)-1:
    bottom_bound = 4
    upper_bound = 0 
else:
    bottom_bound = 3
    upper_bound = 2 
    
rank_list[rank-bottom_bound:rank+upper_bound]

[{'extent': 12102000,
  'iso': 'NOR',
  'name': 'Norway',
  'rate': 13.234,
  'year': 2010},
 {'extent': 380000,
  'iso': 'BRN',
  'name': 'Brunei Darussalam',
  'rate': 0.4,
  'year': 2010},
 {'extent': 498458000,
  'iso': 'BRA',
  'name': 'Brazil',
  'rate': 519,
  'year': 2010},
 {'extent': 242000,
  'iso': 'MRT',
  'name': 'Mauritania',
  'rate': 0.25,
  'year': 2010},
 {'extent': 5820,
  'iso': 'WLF',
  'name': 'Wallis and Futuna',
  'rate': 0.006,
  'year': 2010}]

### Note

Here relative rate (%/year) is calculated by:

    100* (rate / extent)